### Líbrerias necesarias

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import time
import geopandas as gpd
import os
import pyproj
import networkx as nx
import matplotlib.colors as mcolors
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.cm as cmx
import contextily as cx
from tqdm import tqdm
import re
import folium
from IPython.display import display

### Matrices OD

In [2]:
viajes_inferidos_expand = pd.read_csv("C:/Users/JORGE/OneDrive/Desktop/TIC/Base/viajes_inferidos.csv")
viajes_inferidos_expand["viajes_expandido"] = round(viajes_inferidos_expand["viajes_expandido"], 0).astype(int)
rutas_gad = gpd.read_file("C:/Users/JORGE/OneDrive/Desktop/TIC/Rutas_GAD/Red_transporte.shp")
paradas_gad = gpd.read_file("C:/Users/JORGE/OneDrive/Desktop/TIC/Paradas_GAD/Paradas depuradas.shp")
afc_25_completo = pd.read_csv("C:/Users/JORGE/OneDrive/Desktop/TIC/Base/afc_25_completo.csv")

- 05:00–05:59

In [3]:
a = viajes_inferidos_expand[["ParadaInicial", "ParadaEstimDescenso", "viajes_expandido"]][viajes_inferidos_expand["banda_tiempo"]=="05:00–05:59"]
a.sort_values("viajes_expandido", ascending=False).head(20)

,ParadaInicial,ParadaEstimDescenso,viajes_expandido
61830,3627,3390,28
32428,224,518,19
32422,224,364,19
59316,3221,2878,10
60977,3422,3450,9
59231,3221,2189,8
78179,601,1921,8
4677,1211,1921,7
76765,557,1132,7
7964,14,1966,7


In [10]:
# Crear la matriz origen-destino
matriz_od = a.pivot_table(index='ParadaInicial',
                           columns='ParadaEstimDescenso',
                           values='viajes_expandido',
                           aggfunc='sum',
                           fill_value=0)

filas_deseadas = [3627, 224, 3221, 3422, 601, 1211, 557, 14, 632, 94, 819, 2019, 3873, 3953, 441]
columnas_deseadas = [3390, 518, 364, 2878, 3450, 2189, 1921, 1132, 1966, 4138, 2515, 2754, 1772, 1017, 566, 1082]

submatriz = matriz_od.loc[filas_deseadas, columnas_deseadas]

In [11]:
# Mostrar todas las columnas y filas
pd.set_option('display.max_columns', None)  
pd.set_option('display.width', None)    
#pd.set_option('display.max_rows', None)

display(submatriz)

ParadaEstimDescenso,3390,518,364,2878,3450,2189,1921,1132,1966,4138,2515,2754,1772,1017,566,1082
ParadaInicial,,,,,,,,,,,,,,,,
3627,28,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
224,0,19,19,0,0,0,0,0,0,0,0,0,0,0,0,0
3221,0,0,0,10,0,8,0,0,0,0,0,0,0,0,0,0
3422,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,0
601,0,0,0,0,0,0,8,0,0,0,0,0,0,0,0,0
1211,0,0,0,0,0,0,7,4,0,0,6,0,0,0,0,0
557,0,0,0,0,0,0,3,7,0,0,0,0,2,0,0,0
14,0,0,0,0,0,0,0,1,7,0,0,0,0,0,0,0
632,0,0,0,0,0,0,7,0,0,0,0,0,0,0,0,0


- 06:00–07:59

In [12]:
a = viajes_inferidos_expand[["ParadaInicial", "ParadaEstimDescenso", "viajes_expandido"]][viajes_inferidos_expand["banda_tiempo"]=="06:00–07:59"]
a.sort_values("viajes_expandido", ascending=False).head(20)

,ParadaInicial,ParadaEstimDescenso,viajes_expandido
47298,2789,4006,95
49367,2848,1866,93
61831,3627,3390,75
21812,1866,3047,71
15493,1701,2831,69
19189,1790,2831,61
56172,3084,4006,54
52448,2959,4006,53
21781,1866,2848,53
84770,957,1851,53


In [13]:
# Crear la matriz origen-destino
matriz_od = a.pivot_table(index='ParadaInicial',
                           columns='ParadaEstimDescenso',
                           values='viajes_expandido',
                           aggfunc='sum',
                           fill_value=0)

filas_deseadas = [2789, 2848, 3627, 1866, 1701, 1790, 3084, 2959, 957, 4499, 3862, 4547, 4544, 819, 601 , 1721]
columnas_deseadas = [4006, 1866, 3390, 3047, 2831, 2848, 1851, 2484, 1128, 2382, 1724, 1558, 1117]

submatriz = matriz_od.loc[filas_deseadas, columnas_deseadas]

In [14]:
display(submatriz)

ParadaEstimDescenso,4006,1866,3390,3047,2831,2848,1851,2484,1128,2382,1724,1558,1117
ParadaInicial,,,,,,,,,,,,,
2789,95,0,0,0,0,0,0,4,0,0,0,0,0
2848,0,93,0,0,0,0,0,2,0,0,0,0,0
3627,0,0,75,0,0,0,0,0,0,0,0,0,0
1866,0,0,0,71,0,53,0,47,0,0,0,0,0
1701,0,0,0,0,69,0,0,0,0,0,0,0,0
1790,1,0,0,0,61,0,0,0,0,0,0,0,0
3084,54,0,0,0,0,0,0,0,0,0,0,0,0
2959,53,0,0,0,3,0,0,0,0,0,0,0,0
957,0,0,0,0,0,0,53,0,0,0,0,0,0


- 08:00–11:59

In [15]:
a = viajes_inferidos_expand[["ParadaInicial", "ParadaEstimDescenso", "viajes_expandido"]][viajes_inferidos_expand["banda_tiempo"]=="08:00–11:59"]
a.sort_values("viajes_expandido", ascending=False).head(20)

,ParadaInicial,ParadaEstimDescenso,viajes_expandido
73589,4499,2193,126
30365,2189,2317,124
30489,2189,3375,87
71453,4330,2096,76
59233,3221,2189,72
47299,2789,4006,64
78146,601,1558,63
72095,4367,3126,60
57585,3121,4114,60
58230,3135,4114,60


In [16]:
# Crear la matriz origen-destino
matriz_od = a.pivot_table(index='ParadaInicial',
                           columns='ParadaEstimDescenso',
                           values='viajes_expandido',
                           aggfunc='sum',
                           fill_value=0)

filas_deseadas = [4499, 2189, 4330, 3221, 2789, 601, 4367, 3121, 3135, 2437, 2696, 2432, 3375, 2019]
columnas_deseadas = [2193, 2317, 3375, 2096, 2189, 4006, 1558, 3126, 4114, 2685, 1921, 2742, 3221, 2484, 3350, 4330, 581]

submatriz = matriz_od.loc[filas_deseadas, columnas_deseadas]

In [17]:
display(submatriz)

ParadaEstimDescenso,2193,2317,3375,2096,2189,4006,1558,3126,4114,2685,1921,2742,3221,2484,3350,4330,581
ParadaInicial,,,,,,,,,,,,,,,,,
4499,126,29,0,0,0,0,0,0,0,60,0,0,0,57,0,0,0
2189,0,124,87,0,0,0,0,0,0,0,0,0,57,0,0,0,0
4330,0,0,0,76,0,0,0,0,0,0,0,2,0,0,0,0,0
3221,0,13,1,0,72,0,0,0,0,0,0,0,0,3,0,0,0
2789,0,16,0,0,0,64,0,0,0,6,0,0,0,4,0,0,0
601,0,0,0,0,0,0,63,0,0,0,21,0,0,0,0,0,0
4367,2,0,0,0,0,0,0,60,0,0,0,58,0,0,57,2,0
3121,4,0,0,0,0,0,0,0,60,0,1,0,0,3,0,0,0
3135,18,3,0,0,0,17,0,0,60,0,0,0,0,5,0,0,2


- 12:00–13:59

In [18]:
a = viajes_inferidos_expand[["ParadaInicial", "ParadaEstimDescenso", "viajes_expandido"]][viajes_inferidos_expand["banda_tiempo"]=="12:00–13:59"]
a.sort_values("viajes_expandido", ascending=False).head(20)

,ParadaInicial,ParadaEstimDescenso,viajes_expandido
36152,2343,2097,77
17381,1729,1772,55
50076,2879,3368,54
38902,2437,1921,52
3301,1139,581,50
47300,2789,4006,50
76847,557,1819,48
52450,2959,4006,48
49377,2848,2172,47
40101,2484,3006,46


In [19]:
# Crear la matriz origen-destino
matriz_od = a.pivot_table(index='ParadaInicial',
                           columns='ParadaEstimDescenso',
                           values='viajes_expandido',
                           aggfunc='sum',
                           fill_value=0)

filas_deseadas = [2343, 1729, 2879, 2437, 1139, 2789, 557, 2959, 2848, 2484, 544, 4312, 3368, 2830, 3047, 2189, 2306, 2696]
columnas_deseadas = [2097, 1772, 3368, 1921, 581, 4006, 1819, 2172, 3006, 399, 834, 3464, 1676, 2957, 1866, 2317, 2437, 3375]

submatriz = matriz_od.loc[filas_deseadas, columnas_deseadas]

In [20]:
display(submatriz)

ParadaEstimDescenso,2097,1772,3368,1921,581,4006,1819,2172,3006,399,834,3464,1676,2957,1866,2317,2437,3375
ParadaInicial,,,,,,,,,,,,,,,,,,
2343,77,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1729,0,55,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2879,0,0,54,0,0,0,0,0,0,0,0,12,0,0,0,2,0,0
2437,0,0,0,52,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1139,0,0,0,0,50,0,0,0,0,0,0,0,0,0,0,0,0,0
2789,0,0,0,0,0,50,0,0,0,0,0,0,0,0,0,16,0,0
557,0,8,0,8,0,0,48,0,0,0,0,0,5,0,0,2,0,0
2959,0,0,0,0,0,48,0,0,0,0,0,0,0,0,0,13,0,0
2848,0,0,0,4,0,0,0,47,0,0,0,0,0,0,33,0,0,0


- 14:00–16:59

In [21]:
a = viajes_inferidos_expand[["ParadaInicial", "ParadaEstimDescenso", "viajes_expandido"]][viajes_inferidos_expand["banda_tiempo"]=="14:00–16:59"]
a.sort_values("viajes_expandido", ascending=False).head(20)

,ParadaInicial,ParadaEstimDescenso,viajes_expandido
71455,4330,2096,79
47301,2789,4006,73
30367,2189,2317,64
30491,2189,3375,64
71053,4312,834,64
73591,4499,2193,55
21696,1866,2172,52
32821,2255,2096,51
49159,2833,2189,48
72103,4367,3127,47


In [22]:
# Crear la matriz origen-destino
matriz_od = a.pivot_table(index='ParadaInicial',
                           columns='ParadaEstimDescenso',
                           values='viajes_expandido',
                           aggfunc='sum',
                           fill_value=0)

filas_deseadas = [4330, 2789, 2189, 4312, 4499, 1866, 2255, 2833, 4367, 2437, 2959, 2848, 2317]
columnas_deseadas = [2096, 4006, 2317, 3375, 834, 2193, 2172, 2189, 3127, 1921, 2742, 3350, 2484, 1866, 3368, 4330, 2294, 2848]

submatriz = matriz_od.loc[filas_deseadas, columnas_deseadas]

In [23]:
display(submatriz)

ParadaEstimDescenso,2096,4006,2317,3375,834,2193,2172,2189,3127,1921,2742,3350,2484,1866,3368,4330,2294,2848
ParadaInicial,,,,,,,,,,,,,,,,,,
4330,79,0,2,0,0,0,0,0,2,0,2,0,2,0,0,0,0,0
2789,0,73,33,0,0,0,0,0,0,0,0,0,3,0,0,0,40,0
2189,0,0,64,64,0,0,0,0,0,0,0,0,0,0,0,0,9,0
4312,0,0,0,0,64,0,0,0,0,0,0,0,0,0,0,0,0,0
4499,0,0,16,0,0,55,2,0,0,0,0,0,41,0,0,0,3,0
1866,0,0,0,0,0,2,52,0,0,5,0,0,16,0,0,0,0,40
2255,51,0,0,0,0,0,3,0,0,0,2,0,8,0,0,41,0,0
2833,0,0,0,0,0,0,0,48,0,0,0,0,0,0,0,0,0,0
4367,0,0,0,0,0,8,0,0,47,5,46,46,0,0,0,2,0,0


- 17:00–18:59

In [24]:
a = viajes_inferidos_expand[["ParadaInicial", "ParadaEstimDescenso", "viajes_expandido"]][viajes_inferidos_expand["banda_tiempo"]=="17:00–18:59"]
a.sort_values("viajes_expandido", ascending=False).head(20)

,ParadaInicial,ParadaEstimDescenso,viajes_expandido
71054,4312,834,72
35338,2317,3368,70
32219,2232,1954,66
21785,1866,2848,61
49371,2848,1866,59
50051,2879,3222,55
50078,2879,3368,51
72125,4367,3350,50
3303,1139,581,48
20933,1843,953,48


In [25]:
# Crear la matriz origen-destino
matriz_od = a.pivot_table(index='ParadaInicial',
                           columns='ParadaEstimDescenso',
                           values='viajes_expandido',
                           aggfunc='sum',
                           fill_value=0)

filas_deseadas = [4312, 2317, 2232, 1866, 2848, 2879, 4367, 1139, 1843, 3012, 2484, 2306, 1948, 981, 1589, 4547]
columnas_deseadas = [834, 3368, 1954, 2848, 1866, 3222, 3350, 581, 953, 4006, 2957, 4018, 693, 100, 383, 1128]

submatriz = matriz_od.loc[filas_deseadas, columnas_deseadas]

In [26]:
display(submatriz)

ParadaEstimDescenso,834,3368,1954,2848,1866,3222,3350,581,953,4006,2957,4018,693,100,383,1128
ParadaInicial,,,,,,,,,,,,,,,,
4312,72,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2317,0,70,0,0,0,44,0,0,0,3,0,0,0,0,0,0
2232,0,0,66,0,0,0,0,0,0,0,0,0,0,0,0,0
1866,0,0,0,61,0,0,0,0,0,0,0,0,0,0,0,0
2848,0,0,0,0,59,0,0,0,0,0,0,0,0,0,0,0
2879,0,51,0,0,0,55,0,0,0,0,0,0,0,0,0,0
4367,0,0,0,0,0,0,50,0,0,0,0,0,0,0,0,0
1139,0,0,0,0,0,0,0,48,0,0,0,0,42,0,0,0
1843,0,0,0,0,0,0,0,0,48,0,0,0,0,0,0,0


- 19:00–22:59

In [27]:
a = viajes_inferidos_expand[["ParadaInicial", "ParadaEstimDescenso", "viajes_expandido"]][viajes_inferidos_expand["banda_tiempo"]=="19:00–22:59"]
a.sort_values("viajes_expandido", ascending=False).head(20)

,ParadaInicial,ParadaEstimDescenso,viajes_expandido
40104,2484,3006,37
29891,2173,834,36
40140,2484,3082,30
34665,2312,1432,29
40395,2484,4520,27
3322,1139,693,25
39742,2484,1692,25
50052,2879,3222,24
68704,4148,1432,24
40069,2484,2957,24


In [28]:
# Crear la matriz origen-destino
matriz_od = a.pivot_table(index='ParadaInicial',
                           columns='ParadaEstimDescenso',
                           values='viajes_expandido',
                           aggfunc='sum',
                           fill_value=0)

filas_deseadas = [2484, 2173, 2312, 1139, 2879, 4148, 3997, 721, 4367, 2959, 1701]
columnas_deseadas = [3006, 834, 3082, 1432, 4520, 693, 1692, 3222, 2957, 3495, 581, 1680, 3132, 1721, 2954, 1190, 2016, 3999]

submatriz = matriz_od.loc[filas_deseadas, columnas_deseadas]

In [29]:
display(submatriz)

ParadaEstimDescenso,3006,834,3082,1432,4520,693,1692,3222,2957,3495,581,1680,3132,1721,2954,1190,2016,3999
ParadaInicial,,,,,,,,,,,,,,,,,,
2484,37,0,30,0,27,0,25,21,24,0,0,0,7,0,22,0,0,19
2173,0,36,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2312,0,0,0,29,0,0,0,0,0,0,0,0,0,14,0,0,19,0
1139,0,0,0,0,0,25,0,0,0,0,23,0,0,0,0,0,0,0
2879,0,0,0,0,0,0,0,24,0,0,0,0,0,0,0,0,0,0
4148,0,0,0,24,0,0,0,0,0,0,0,0,0,0,0,0,6,0
3997,0,0,0,0,0,0,0,0,0,24,0,0,0,0,0,0,0,0
721,0,0,0,0,0,0,0,0,0,0,0,23,0,0,0,0,0,0
4367,0,0,0,0,0,0,0,0,0,3,0,0,23,0,0,0,0,0


### Gráficos

- Mapa de líneas de flujo

In [3]:
# Asegurar que ambas columnas sean string (texto)
viajes_inferidos_expand['ParadaInicial'] = viajes_inferidos_expand['ParadaInicial'].astype(int)
viajes_inferidos_expand['ParadaEstimDescenso'] = viajes_inferidos_expand['ParadaEstimDescenso'].astype(int)

# Renombrar columnas en el catálogo de paradas para evitar confusión al unir
paradas_origen = paradas_gad[['ID-PARADA', 'LATITUD', 'LONGITUD']].rename(columns={
    'ID-PARADA': 'ParadaInicial',
    'LATITUD': 'lat_origen',
    'LONGITUD': 'lon_origen'
})

paradas_destino = paradas_gad[['ID-PARADA', 'LATITUD', 'LONGITUD']].rename(columns={
    'ID-PARADA': 'ParadaEstimDescenso',
    'LATITUD': 'lat_destino',
    'LONGITUD': 'lon_destino'
})

paradas_origen['ParadaInicial'] = paradas_origen['ParadaInicial'].astype(int)
paradas_destino['ParadaEstimDescenso'] = paradas_destino['ParadaEstimDescenso'].astype(int)

# Unir coordenadas de ORIGEN
viajes = viajes_inferidos_expand.merge(paradas_origen, on='ParadaInicial', how='left')

# Unir coordenadas de DESTINO
viajes = viajes.merge(paradas_destino, on='ParadaEstimDescenso', how='left')

In [ ]:
viajes.sort_values("viajes_expandido", ascending=False)

,ParadaInicial,ParadaEstimDescenso,banda_tiempo,viajes_inferidos,f_it,viajes_expandido,lat_origen,lon_origen,lat_destino,lon_destino
0,10,12,12:00–13:59,1,1.000000,1.000000,-2.779586,-78.977922,-2.780122,-78.966638
1,10,14,08:00–11:59,1,1.000000,1.000000,-2.779586,-78.977922,-2.783975,-78.962598
2,10,19,06:00–07:59,3,1.000000,3.000000,-2.779586,-78.977922,-2.788914,-78.962195
3,100,59,17:00–18:59,1,1.000000,1.000000,-2.815876,-78.995378,-2.804702,-78.993203
4,1001,1049,08:00–11:59,1,1.631148,1.631148,-2.880684,-79.064369,-2.881484,-79.061744
...,...,...,...,...,...,...,...,...,...,...
85258,993,713,12:00–13:59,1,1.444444,1.444444,-2.879822,-78.967163,-2.870979,-78.965943
85259,993,945,06:00–07:59,1,1.538462,1.538462,-2.879822,-78.967163,-2.879256,-78.981692
85260,993,945,08:00–11:59,1,1.363636,1.363636,-2.879822,-78.967163,-2.879256,-78.981692
85261,993,945,12:00–13:59,4,1.444444,5.777778,-2.879822,-78.967163,-2.879256,-78.981692


In [31]:
aw = viajes[viajes["banda_tiempo"]=="17:00–18:59"].groupby("ParadaInicial").count()
aw[["ParadaEstimDescenso"]].sort_values("ParadaEstimDescenso", ascending=False)

,ParadaEstimDescenso
ParadaInicial,
1701,141
2484,139
4499,125
2317,122
4330,104
...,...
80,1
93,1
100,1


In [ ]:
#viajes.to_excel(os.path.join(ruta, "viajes.xlsx"), index=False)

In [ ]:
# Crear carpeta de salida
os.makedirs("mapas_por_banda", exist_ok=True)

# Asegurar coordenadas numéricas
viajes[['lat_origen', 'lon_origen', 'lat_destino', 'lon_destino']] = viajes[
    ['lat_origen', 'lon_origen', 'lat_destino', 'lon_destino']
].astype(float)

# Proyección de WGS84 a Web Mercator
project = pyproj.Transformer.from_crs(4326, 3857, always_xy=True).transform

subset = viajes.copy()

# Filtrar self-loops
subset = subset[subset['ParadaInicial'] != subset['ParadaEstimDescenso']]

subset_agrupado = subset.groupby(
    ['ParadaInicial', 'ParadaEstimDescenso', 'lat_origen', 'lon_origen', 'lat_destino', 'lon_destino'], as_index=False
)['viajes_expandido'].sum()

# Crear grafo dirigido
G = nx.DiGraph()
for _, row in subset_agrupado.iterrows():
    G.add_node(row['ParadaInicial'], pos=(row['lon_origen'], row['lat_origen']))
    G.add_node(row['ParadaEstimDescenso'], pos=(row['lon_destino'], row['lat_destino']))
    G.add_edge(row['ParadaInicial'], row['ParadaEstimDescenso'], weight=row['viajes_expandido'])

# for u, v, attrs in sorted(G.edges(data=True), key=lambda x: x[2]['weight'], reverse=True)[:10]:
#     print(f"De {u} a {v}, peso: {attrs['weight']}")


pos = nx.get_node_attributes(G, 'pos')
pos_mercator = {k: project(*v) for k, v in pos.items()}

# Crear GeoDataFrame de nodos con tamaños
nodos_df = pd.DataFrame.from_dict(pos_mercator, orient='index', columns=['x', 'y'])
nodos_df['node'] = nodos_df.index

gdf_nodos = gpd.GeoDataFrame(nodos_df, geometry=gpd.points_from_xy(nodos_df['x'], nodos_df['y']), crs='EPSG:3857')

# Obtener pesos para edges
edges = sorted(G.edges(data=True), key=lambda x: x[2]['weight'])
weights = [d['weight'] for (_, _, d) in edges]

w_min, w_max = min(weights), max(weights)

norm = mcolors.Normalize(vmin=w_min, vmax=w_max)

custom_gradient = LinearSegmentedColormap.from_list(
    'custom_gradient',
    [
        (0.0, "#e2ea74"),  # amarillo claro (inicio, por ejemplo viajes=0)
        (0.5, "#fca935"),  # naranja (por ejemplo viajes~15)
        (0.8, "#f93a1d"),  # rojo-naranja (viajes~24)
        (1.0, "#ff0000")   # rojo puro (viajes=30)
    ],
    N=256
)
scalar_map = cmx.ScalarMappable(norm=norm, cmap=custom_gradient)

# Crear figura
fig, ax = plt.subplots(figsize=(10, 10))

# Dibujar edges con color y grosor sincronizados
for (u, v, d) in edges:
    weight = d['weight']
    w_norm = norm(weight)
    color = scalar_map.to_rgba(weight)
    linewidth = 1 + w_norm

    x_vals = [pos_mercator[u][0], pos_mercator[v][0]]
    y_vals = [pos_mercator[u][1], pos_mercator[v][1]]

    ax.plot(x_vals, y_vals, color=color, linewidth=linewidth, alpha=0.7)

    # # Agregar texto solo si el peso es alto
    # if weight > 100:
    #     x_text = (x_vals[0] + x_vals[1]) / 2
    #     y_text = (y_vals[0] + y_vals[1]) / 2
    #     ax.text(
    #         x_text, y_text,
    #         f"{int(u)} → {int(v)}",
    #         fontsize=6,
    #         color='black',
    #         ha='center',
    #         va='center',
    #         bbox=dict(boxstyle="round,pad=0.2", fc="white", ec="black", lw=0.5, alpha=0.7)
    #     )

# Añadir mapa base
cx.add_basemap(ax, source=cx.providers.OpenStreetMap.Mapnik)

# Añadir leyenda de colores
cbar = plt.colorbar(scalar_map, ax=ax, orientation='vertical', shrink=0.5)
cbar.set_label("Número de viajes")

# Dibujar nodos con tamaños variables
gdf_nodos.plot(ax=ax, color='black', alpha=1, markersize=5)

# Título y guardar imagen
ax.set_title(f"Mapa de flujo", fontsize=14)
ax.axis("off")
archivo = f"C:/Users/JORGE/OneDrive/Desktop/TIC/Mapas_por_banda/flujo_{"completo_aux"}.png"
plt.savefig(archivo, dpi=300, bbox_inches='tight')
plt.close()

In [5]:
# Crear carpeta de salida
os.makedirs("mapas_por_banda", exist_ok=True)

# Asegurar coordenadas numéricas
viajes[['lat_origen', 'lon_origen', 'lat_destino', 'lon_destino']] = viajes[
    ['lat_origen', 'lon_origen', 'lat_destino', 'lon_destino']
].astype(float)

# Proyección de WGS84 a Web Mercator
project = pyproj.Transformer.from_crs(4326, 3857, always_xy=True).transform

# Recorrer cada franja horaria
for banda in tqdm(viajes['banda_tiempo'].unique(), desc="Procesando bandas"):
    subset = viajes[viajes['banda_tiempo'] == banda].copy()
    
    # Filtrar self-loops
    subset = subset[subset['ParadaInicial'] != subset['ParadaEstimDescenso']]

    # Filtrar si deseas solo los más frecuentes
    #subset = subset.sort_values('viajes_expandido', ascending=False).head(500)

    # Crear grafo dirigido
    G = nx.DiGraph()
    for _, row in subset.iterrows():
        G.add_node(row['ParadaInicial'], pos=(row['lon_origen'], row['lat_origen']))
        G.add_node(row['ParadaEstimDescenso'], pos=(row['lon_destino'], row['lat_destino']))
        G.add_edge(row['ParadaInicial'], row['ParadaEstimDescenso'], weight=row['viajes_expandido'])

    pos = nx.get_node_attributes(G, 'pos')
    pos_mercator = {k: project(*v) for k, v in pos.items()}

    # Crear GeoDataFrame de nodos con tamaños
    nodos_df = pd.DataFrame.from_dict(pos_mercator, orient='index', columns=['x', 'y'])
    nodos_df['node'] = nodos_df.index
    # nodos_df['grado'] = nodos_df['node'].apply(lambda n: G.degree(n, weight='weight'))
    # nodos_df['size'] = np.interp(nodos_df['grado'], (nodos_df['grado'].min(), nodos_df['grado'].max()), (30, 300))

    gdf_nodos = gpd.GeoDataFrame(nodos_df, geometry=gpd.points_from_xy(nodos_df['x'], nodos_df['y']), crs='EPSG:3857')

    # Obtener pesos para edges
    edges = sorted(G.edges(data=True), key=lambda x: x[2]['weight'])
    weights = [d['weight'] for (_, _, d) in edges]
    if not weights:
        continue

    w_min, w_max = min(weights), max(weights)
    norm = mcolors.Normalize(vmin=w_min, vmax=w_max)
    
    custom_gradient = LinearSegmentedColormap.from_list(
    'custom_gradient',
    [
        (0.0, "#e2ea74"),  # amarillo claro (inicio, por ejemplo viajes=0)
        (0.5, "#fca935"),  # naranja (por ejemplo viajes~15)
        (0.8, "#f93a1d"),  # rojo-naranja (viajes~24)
        (1.0, "#ff0000")   # rojo puro (viajes=30)
    ],
    N=256
)
    scalar_map = cmx.ScalarMappable(norm=norm, cmap=custom_gradient)

    # Crear figura
    fig, ax = plt.subplots(figsize=(10, 10))

    # Dibujar edges con color y grosor sincronizados
    for (u, v, d) in edges:
        weight = d['weight']
        w_norm = norm(weight)
        color = scalar_map.to_rgba(weight)
        linewidth = 1 + w_norm

        x_vals = [pos_mercator[u][0], pos_mercator[v][0]]
        y_vals = [pos_mercator[u][1], pos_mercator[v][1]]

        ax.plot(x_vals, y_vals, color=color, linewidth=linewidth, alpha=0.7)

    # Añadir mapa base
    cx.add_basemap(ax, source=cx.providers.OpenStreetMap.Mapnik)

    # Añadir leyenda de colores
    cbar = plt.colorbar(scalar_map, ax=ax, orientation='vertical', shrink=0.5)
    cbar.set_label("Número de viajes")

    # Dibujar nodos con tamaños variables
    gdf_nodos.plot(ax=ax, color='black', alpha=1, markersize=5)

    # Título y guardar imagen
    ax.set_title(f"Mapa de flujo - Banda: {banda}", fontsize=14)
    ax.axis("off")
    nombre_banda = re.sub(r'\W+', '_', banda)
    archivo = f"C:/Users/JORGE/OneDrive/Desktop/TIC/Mapas_por_banda/flujo_{nombre_banda}.png"
    plt.savefig(archivo, dpi=300, bbox_inches='tight')
    plt.close()

Procesando bandas:   0%|          | 0/7 [00:00<?, ?it/s]

Procesando bandas: 100%|██████████| 7/7 [02:01<00:00, 17.31s/it]


- Lineas más usadas

In [10]:
# Filtro
linea_objetivo = '100'  # Código de línea a graficar
linename_shp = '100'     # Código de línea en el shapefile del GAD

# Paradas de la linea
paradas_linea = (
    afc_25_completo[afc_25_completo['NombreLinea'] == linea_objetivo]
    .dropna(subset=['LATITUD', 'LONGITUD', 'SECUENCIA_GLOBAL'])
    .drop_duplicates(subset='CodigoParada')
    .sort_values('SECUENCIA_GLOBAL')
    .copy()
)

# Extraer la linea específica 
rutas_filtradas = rutas_gad[rutas_gad['linename'] == linename_shp]

# Reproyectar a EPSG:4326 si es necesario
if rutas_filtradas.crs and rutas_filtradas.crs.to_string() != 'EPSG:4326':
    rutas_filtradas = rutas_filtradas.to_crs(epsg=4326)

# Mapa base centrado en Cuenca 
centro_cuenca = [-2.90055, -79.00453]
mapa = folium.Map(location=centro_cuenca, zoom_start=13)

# Agregar puntos de paradas
for _, fila in paradas_linea.iterrows():
    folium.CircleMarker(
        location=[fila['LATITUD'], fila['LONGITUD']],
        radius=4,
        color='darkblue',
        fill=True,
        fill_color='darkblue',
        fill_opacity=0.8,
        popup=f"Parada: {fila['CodigoParada']}<br>Secuencia: {fila['SECUENCIA_GLOBAL']:.0f}"
    ).add_to(mapa)

    # # Etiqueta con número de secuencia
    # folium.Marker(
    #     location=[fila['LATITUD'], fila['LONGITUD']],
    #     icon=folium.DivIcon(html=f"""
    #         <div style="font-size: 12pt; color: black; text-align: center;">
    #             {int(fila['SECUENCIA_GLOBAL'])}
    #         </div>
    #     """)
    # ).add_to(mapa)

# Dibujar la línea oficial desde rutas_gad 
for _, fila in rutas_filtradas.iterrows():
    coords = [(y, x) for x, y in fila['geometry'].coords]
    folium.PolyLine(
        locations=coords,
        color='darkblue',
        weight=4,
        opacity=0.8,
        tooltip=f"Línea {fila['linename']} - {fila['sentido']}"
    ).add_to(mapa)

# Guardar o mostrar el mapa 
mapa.save("C:/Users/JORGE/OneDrive/Desktop/TIC/Mapas_lineas/Línea_24.html")
mapa